In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import random
import torch
import datetime


import sys
sys.path.append('../src/utils') 
from common import get_accuracy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SAVE_PATH = "/home/s/python_progs/DL_homeworks/deep-metric-ts-clustering/notebooks/model"

In [2]:
# Select diverse stocks from different sectors
tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'AMZN', 'JPM', 'JNJ', 'XOM', 'NVDA', 'WMT']
ticker_to_class = {ticker: i for i, ticker in enumerate(tickers)}

data = yf.download(tickers, start="2020-01-01", end="2024-01-01")['Close']
data.dropna(inplace=True)
print(data)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed

Ticker            AAPL        AMZN       GOOGL         JNJ         JPM  \
Date                                                                     
2020-01-02   72.620827   94.900497   68.108376  126.055145  120.733566   
2020-01-03   71.914803   93.748497   67.752075  124.595749  119.140327   
2020-01-06   72.487846   95.143997   69.557945  124.440338  119.045586   
2020-01-07   72.146942   95.343002   69.423584  125.200233  117.021744   
2020-01-08   73.307503   94.598503   69.917732  125.182945  117.934608   
...                ...         ...         ...         ...         ...   
2023-12-22  192.192551  153.419998  140.816757  149.492691  161.660004   
2023-12-26  191.646561  153.410004  140.846634  150.146591  162.616074   
2023-12-27  191.745834  153.339996  139.702087  150.348541  163.591431   
2023-12-28  192.172714  153.380005  139.562744  150.569702  164.460587   
2023-12-29  191.130325  151.940002  139.025330  150.723572  164.267441   

Ticker            MSFT       NVDA    

In [3]:
def create_windows(series, window_size=30, step=1):
    windows = []
    for i in range(0, len(series) - window_size + 1, step):
        window = series[i:i + window_size].values
        windows.append(window)
    return windows


def create_dataset(windows):
    dataset = []
    for window in windows:

        dataset.append((window['window'], ticker_to_class[window['ticker']]))

    return dataset


In [4]:
window_size = 30
step = 5  # reduces overlap between samples
stock_windows = []  # Dictionary of ticker → list of windows

for ticker in tickers:
    for window in create_windows(data[ticker], window_size, step):
        stock_windows.append({
                'ticker': ticker,
                'window': window
            })
print(stock_windows[0])

{'ticker': 'AAPL', 'window': array([72.62082672, 71.91480255, 72.48784637, 72.14694214, 73.30750275,
       74.86462402, 75.03387451, 76.63691711, 75.60206604, 75.27807617,
       76.22105408, 77.06489563, 76.54263306, 76.8158493 , 77.18579102,
       76.96333313, 74.70021057, 76.81343842, 78.4213028 , 78.30767822,
       74.83560181, 74.63008881, 77.09391022, 77.72257233, 78.63166809,
       77.56285858, 77.9312439 , 77.4610672 , 79.30059814, 78.73588562])}


In [5]:
dataset = create_dataset(stock_windows)

In [6]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split

class StockDataset(Dataset):
    def __init__(self, stock_windows):
        self.stock_windows = stock_windows
        self.labels = [label for _, label in stock_windows]  # Extract and store labels

    def __len__(self):
        return len(self.stock_windows)

    def __getitem__(self, idx):
        x, label = self.stock_windows[idx]
        x = torch.tensor(x, dtype=torch.float32).unsqueeze(0)  # shape [1, 30]
        label = torch.tensor(label, dtype=torch.float32)
        return x, label

dataset = StockDataset(dataset)


train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
val_size = len(dataset) - train_size


train_dataset, val_dataset = random_split(dataset, [train_size, val_size])



In [7]:
import torch.nn as nn


class Encoder(nn.Module):
    def __init__(self, input_dim, encoder_type, embedding_dim=128):
        super().__init__()

        self.encoder_type = encoder_type
        self.embedding_dim = embedding_dim


        if encoder_type=="CNN":
            self.conv_layers = nn.Sequential(
                nn.Conv1d(1, 64, kernel_size=3, padding=1),   # [30] → [30]
                nn.ReLU(),
                nn.Conv1d(64, 64, kernel_size=3, padding=1),  # [30] → [30]
                nn.ReLU(),
                nn.MaxPool1d(2),                              # [30] → [15]

                nn.Conv1d(64, 128, kernel_size=3, padding=1), # [15] → [15]
                nn.ReLU(),
                nn.Conv1d(128, 128, kernel_size=3, padding=1),# [15] → [15]
                nn.ReLU(),
                nn.MaxPool1d(2),                              # [15] → [7]

                nn.Conv1d(128, 256, kernel_size=3, padding=1),# [7] → [7]
                nn.ReLU(),
                nn.Conv1d(256, 256, kernel_size=3, padding=1),# [7] → [7]
                nn.ReLU(),
                nn.MaxPool1d(2),                              # [7] → [3]
            )

            self.fc = nn.Linear(256 * 3, embedding_dim)


        if encoder_type=="GRU":
            self.num_layers = 5
            self.hidden_size = 256
            self.gru = nn.GRU(1, self.hidden_size, self.num_layers, batch_first=True)
            self.fc = nn.Linear(self.hidden_size, self.embedding_dim)

        if encoder_type=="LSTM":    
            self.num_layers = 5
            self.hidden_size = 256
            self.lstm = nn.LSTM(1, self.hidden_size, self.num_layers, batch_first=True)
            self.fc = nn.Linear(self.hidden_size, self.embedding_dim)

    def forward(self, x):
        if self.encoder_type=="CNN":
            # x = x.unsqueeze(1)  # add channel dim: (batch_size, 1, seq_len)
            x = self.conv_layers(x)
            x = x.view(x.size(0), -1)  # flatten
            x = self.fc(x)
        
        if self.encoder_type=="GRU":
            x = x.reshape(-1, window_size, 1)

            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

            x, _ = self.gru(x, h0)
            x = x[:, -1, :]
            x = self.fc(x)
            x = nn.functional.normalize(x, p=2, dim=1)

        if self.encoder_type=="LSTM":
            x = x.reshape(-1, window_size, 1)

            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
            c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)


            x, _ = self.lstm(x, (h0,c0))
            x = x[:, -1, :]
            x = self.fc(x)
            x = nn.functional.normalize(x, p=2, dim=1)



        return x




In [ ]:

from torch.utils.data import DataLoader
from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

distance = distances.CosineSimilarity()
reducer = reducers.ThresholdReducer(low=0)
mining_func = miners.TripletMarginMiner(margin=0.2, distance=distance, type_of_triplets="semihard")
loss_func = losses.TripletMarginLoss(margin=0.2, distance=distance, reducer=reducer)

# model = Encoder(window_size, 'CNN')
# model = Encoder(window_size, 'LSTM')
model = Encoder(window_size, 'GRU')
# model = Encoder(window_size, 'TRANSFORMER')

model.to(device)

Encoder(
  (lstm): LSTM(1, 256, num_layers=5, batch_first=True)
  (fc): Linear(in_features=256, out_features=128, bias=True)
)

In [ ]:
import torch
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
import torch.optim as optim
from tqdm import tqdm
import json

batch_size = 512

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
history = {"train": [], "val": [], "best_accuracy": 0.0}

scaler = GradScaler()  # Helps manage fp16 gradients

optimizer = optim.Adam(model.parameters(), lr=1e-3)
epochs = 1000

for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for x, label in train_loader:
        x, label = x.to(device), label.to(device)

        optimizer.zero_grad()

        with autocast(dtype=torch.float16, device_type=device.type): 
            embeddings = model(x)
            indices_tuple = mining_func(embeddings, label)
            loss = loss_func(embeddings, label, indices_tuple)

        # Backpropagation with scaled gradients
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        epoch_loss += loss.item()




    if epoch %10 == 0:
        # model validation
        model.eval()
        avg_loss = epoch_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
        
        with torch.no_grad():
            # as all embeddings need to be stored in memory
            # you can set DEVICE = torch.device('cpu') in case gpu memory overflow occurs
            accuracy = get_accuracy(val_dataset, train_dataset, model, device.type)
            
            history["val"].append({"epoch": epoch, "accuracy": accuracy})
            print(f"Val accuracy: {accuracy}")
            
            # save model
            torch.save(model.state_dict(), f"{SAVE_PATH}/model_latest_{model.encoder_type}.pth")
            
            if accuracy >= history["best_accuracy"]:
                history["best_accuracy"] = accuracy
                torch.save(model.state_dict(), f"{SAVE_PATH}/model_best_{model.encoder_type}.pth")

with open(f"history_{model.encoder_type}_{datetime.datetime.now()}.json", "w") as f:
    f.write(json.dumps(history))

Epoch 1/1000, Loss: 0.1852


100%|██████████| 49/49 [00:00<00:00, 282.21it/s]


Val accuracy: 0.4897959183673469
Epoch 11/1000, Loss: 0.1139


100%|██████████| 49/49 [00:00<00:00, 383.04it/s]


Val accuracy: 0.4642857142857143
Epoch 21/1000, Loss: 0.1113


...

Val accuracy: 0.7295918367346939
Epoch 981/1000, Loss: 0.0805


100%|██████████| 49/49 [00:00<00:00, 268.21it/s]


Val accuracy: 0.8112244897959183
Epoch 991/1000, Loss: 0.0899


100%|██████████| 49/49 [00:00<00:00, 259.30it/s]


Val accuracy: 0.8163265306122449


In [ ]:
model.load_state_dict(torch.load("/home/s/python_progs/DL_homeworks/deep-metric-ts-clustering/notebooks/model/model_GRU_absolute_best.pth", map_location=torch.device("cuda")))
model.to("cuda")

model.eval()

with torch.no_grad():
    # as all embeddings need to be stored in memory
    # you can set DEVICE = torch.device('cpu') in case gpu memory overflow occurs
    accuracy = get_accuracy(val_dataset, train_dataset, model, device.type)
    
    print(f"Val accuracy: {accuracy}")

RuntimeError: Error(s) in loading state_dict for Encoder:
	Missing key(s) in state_dict: "lstm.weight_ih_l0", "lstm.weight_hh_l0", "lstm.bias_ih_l0", "lstm.bias_hh_l0", "lstm.weight_ih_l1", "lstm.weight_hh_l1", "lstm.bias_ih_l1", "lstm.bias_hh_l1", "lstm.weight_ih_l2", "lstm.weight_hh_l2", "lstm.bias_ih_l2", "lstm.bias_hh_l2", "lstm.weight_ih_l3", "lstm.weight_hh_l3", "lstm.bias_ih_l3", "lstm.bias_hh_l3", "lstm.weight_ih_l4", "lstm.weight_hh_l4", "lstm.bias_ih_l4", "lstm.bias_hh_l4". 
	Unexpected key(s) in state_dict: "gru.weight_ih_l0", "gru.weight_hh_l0", "gru.bias_ih_l0", "gru.bias_hh_l0", "gru.weight_ih_l1", "gru.weight_hh_l1", "gru.bias_ih_l1", "gru.bias_hh_l1", "gru.weight_ih_l2", "gru.weight_hh_l2", "gru.bias_ih_l2", "gru.bias_hh_l2", "gru.weight_ih_l3", "gru.weight_hh_l3", "gru.bias_ih_l3", "gru.bias_hh_l3", "gru.weight_ih_l4", "gru.weight_hh_l4", "gru.bias_ih_l4", "gru.bias_hh_l4". 

2025-05-27 22:01:46.628948
